In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import os
import io
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import pandas as pd

Base= '/content/drive/MyDrive/Fum/term9/Project_phase2'
lable= os.path.join(Base,'lable.csv')
trainCSV=os.path.join(Base ,'train.csv')
validationCSV=os.path.join(Base ,'validation.csv')
testCSV= os.path.join(Base,'test.csv')
df=pd.read_csv(lable)
train_df,temp_df=train_test_split(df,test_size=0.3,random_state=42)
val_df,test_df= train_test_split(temp_df,test_size=0.5 ,random_state=42)
train_df.to_csv(trainCSV,index=False)
val_df.to_csv(validationCSV,index=False)
test_df.to_csv(testCSV ,index=False)

In [ ]:
!cp -r /content/drive/MyDrive/Fum/term9/Project_phase2/* /content

In [ ]:
!pip install datasets

In [ ]:
!rm -rf ~/.cache/huggingface/datasets

In [ ]:
!pip install huggingface_hub pyarrow pandas

In [ ]:
!git lfs install
!git clone https://huggingface.co/datasets/hezarai/persian-license-plate-v1


Git LFS initialized.
Cloning into 'persian-license-plate-v1'...
remote: Enumerating objects: 95, done.
remote: Total 95 (delta 0), reused 0 (delta 0), pack-reused 95 (from 1)
Unpacking objects: 100% (95/95), 138.43 KiB | 833.00 KiB/s, done.


In [ ]:
!ls ./persian-license-plate-v1/data/

test-00000-of-00001.parquet   validation-00000-of-00001.parquet
train-00000-of-00001.parquet


In [ ]:
import pandas as pd
df= pd.read_parquet('./persian-license-plate-v1/data/train-00000-of-00001.parquet')
print(df.columns)

Index(['image_path', 'label'], dtype='object')


In [ ]:
import json
import os
from PIL import Image
import io
import torch
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import pandas as pd
from tqdm import tqdm

In [ ]:
train_transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64,128)),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2),
    transforms.GaussianBlur(kernel_size=5,sigma=0.5),
    transforms.RandomAffine(degrees=0,shear=5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])
val_transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

In [ ]:

dataset_folder='./persian-license-plate-v1'  
vocab_path='/content/vocab.json' 
trainP=os.path.join(dataset_folder,'data','train-00000-of-00001.parquet')
valP=os.path.join(dataset_folder,'data','validation-00000-of-00001.parquet')
testP=os.path.join(dataset_folder,'data','test-00000-of-00001.parquet')
with open(vocab_path,'r',encoding='utf-8') as f:
    vocab=json.load(f)

char_map={
    '۰': '0','۱': '1','۲': '2'
    ,'۳': '3','۴': '4',
    '۵': '5','۶': '6','۷': '7','۸': '8'
    ,'۹': '9'
}
train_df=pd.read_parquet(trainP)
val_df=pd.read_parquet(valP)
test_df=pd.read_parquet(testP)

def map_and_filter_labels(label):
    mapped_label=''.join(char_map.get(char,char) for char in label)
    return all(char in vocab for char in mapped_label),mapped_label

for df,name in [(train_df,'train'),(val_df,'validation'),(test_df,'test')]:
    valid_data=[]
    for idx,row in tqdm(df.iterrows(),total=len(df),desc=f"پردازش لیبل‌های {name}"):
        is_valid,mapped_label=map_and_filter_labels(row['label'])
        if is_valid:
            row=row.copy() 
            row['label']=mapped_label  
            valid_data.append(row)
    df=pd.DataFrame(valid_data)
    if name=='train':
        train_df=df
    elif name == 'validation':
        val_df=df
    else:
        test_df=df
if len(train_df) == 0:
    raise ValueError("vocab ERR")
if len(val_df) == 0:
    print("No data")
if len(test_df) == 0:
    print("NO data")

class PlateDataset(Dataset):
    def __init__(self,df,transform=None):
        self.df=df
        self.transform=transform

    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        row=self.df.iloc[idx]
        if 'image' in self.df.columns:
            image_data=row['image']
            if isinstance(image_data,bytes):
                image=Image.open(io.BytesIO(image_data))
            else:
                image=Image.fromarray(image_data)
        else:
            raise ValueError("ERR")
        image=image.convert('L') 
        label=row['label']
        label_encoded=[vocab[char] for char in label]
        label_encoded=torch.tensor(label_encoded,dtype=torch.int64)

        if self.transform:
            image=self.transform(image)

        return image,label_encoded

train_dataset= PlateDataset(train_df,transform=train_transform)
val_dataset=PlateDataset(val_df,transform=val_transform)
test_dataset= PlateDataset(test_df,transform=val_transform)

def collate_fn(batch):
    IMGE,lable=zip(*batch)
    IMGE= torch.stack(IMGE)
    return IMGE,lable

trainL= DataLoader(train_dataset,batch_size=8,shuffle=True,num_workers=2,collate_fn=collate_fn)
valL= DataLoader(val_dataset,batch_size=8,shuffle=False,num_workers=2,collate_fn=collate_fn)
testL= DataLoader(test_dataset,batch_size=8,shuffle=False,num_workers=2,collate_fn=collate_fn)


In [ ]:
import pandas as pd
df=pd.read_parquet('./persian-license-plate-v1/data/train-00000-of-00001.parquet')
print(df.columns)
print(df.head())

Index(['image_path', 'label'], dtype='object')
                                          image_path     label
0  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...  ۱۴س۵۶۹۱۴
1  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...  ۶۶د۸۶۸۲۴
2  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...  ۴۶د۸۵۷۱۰
3  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...  ۷۲ص۳۹۸۱۰
4  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...  ۶۴س۸۵۴۸۸


In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size=8
learning_rate=0.0002
weight_decay=0.0001
max_epochs= 200
patience=20
best_model_path="/content/drive/MyDrive/Fum/term9/Project_phase2/ocr_v3_p20.pth"
finetuned_model_path="/content/drive/MyDrive/Fum/term9/Project_phase2/ocr_finetuned.pth"
vocab_path='/content/vocab.json'

In [ ]:
class CRNN(nn.Module):
    def __init__(self,num_classes,hidden_size=256):
        super(CRNN,self).__init__()

        self.cnn=nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,padding=1),
            PrintShape("Conv1"),
            nn.ReLU(True),
            PrintShape("ReLU1"),
            nn.Dropout2d(0.3),
            PrintShape("Dropout1"),
            nn.MaxPool2d(kernel_size=2,stride=2),
            PrintShape("MaxPool1"),
            nn.Conv2d(64,128,kernel_size=3,padding=1),
            PrintShape("Conv2"),
            nn.ReLU(True),
            PrintShape("ReLU2"),
            nn.Dropout2d(0.3),
            PrintShape("Dropout2"),
            nn.MaxPool2d(kernel_size=2,stride=2),
            PrintShape("MaxPool2"),
            nn.Conv2d(128,256,kernel_size=3,padding=1),
            PrintShape("Conv3"),
            nn.ReLU(True),
            PrintShape("ReLU3"),
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            PrintShape("Conv4"),
            nn.ReLU(True),
            PrintShape("ReLU4"),
            nn.Dropout2d(0.3),
            PrintShape("Dropout3"),
            nn.MaxPool2d(kernel_size=2,stride=2),
            PrintShape("MaxPool3"),
            nn.Conv2d(256,512,kernel_size=3,padding=1),
            PrintShape("Conv5"),
            nn.ReLU(True),
            PrintShape("ReLU5"),
            nn.BatchNorm2d(512),
            PrintShape("BatchNorm"),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            PrintShape("Conv6"),
            nn.ReLU(True),
            PrintShape("ReLU6"),
            nn.Dropout2d(0.3),
            PrintShape("Dropout4"),
            nn.MaxPool2d(kernel_size=(4,1)),
            PrintShape("MaxPool4"),
            nn.Conv2d(512,512,kernel_size=2,stride=1),
            PrintShape("Conv7"),
            nn.ReLU(True),
            PrintShape("ReLU7")
        )

        self.dropout_rnn=nn.Dropout(0.2)




        self.rnn=nn.Sequential(
            nn.LSTM(input_size=512,hidden_size=hidden_size,num_layers=2,
                    bidirectional=True,batch_first=True)
        )

        self.fc=nn.Linear(hidden_size * 2,num_classes)

    def forward(self,x):
        conv=self.cnn(x)
        batch_size,channels,height,width=conv.size()
        conv=conv.squeeze(2)
        conv=conv.permute(0,2,1)
        conv=self.dropout_rnn(conv)
        rnn_out,_=self.rnn(conv)
        out=self.fc(rnn_out)
        return out


In [ ]:
def validate_epoch(M,loader,criterion):
    M.eval()
    total_loss=0
    with torch.no_grad():
        for IMGE,lable in tqdm(loader,desc="ارزیابی"):
            IMGE=IMGE.to(device)
            max_len=max([len(label) for label in lable])
            padded_labels=torch.zeros(len(lable),max_len,dtype=torch.long).to(device)
            for i,label in enumerate(lable):
                padded_labels[i,:len(label)]=label
            outputs=M(IMGE).log_softmax(2)

            
            batch_size=IMGE.size(0)
            input_lengths=torch.full((batch_size,),outputs.size(1),dtype=torch.long)
            target_lengths=torch.tensor([len(label) for label in lable],dtype=torch.long)
            loss=criterion(outputs.permute(1,0,2),padded_labels,input_lengths,target_lengths)
            total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
def compute_accuracy(M,loader,reverse_vocab):
    M.eval()
    total_correctP=0
    total_correctC=0
    totalC= 0
    totalS= 0
    with torch.no_grad():
        for IMGE,lable in loader:
            IMGE= IMGE.to(device)
            outputs= M(IMGE).softmax(2)
            decodedP=decode_predictions(outputs,reverse_vocab)
            for i in range(len(lable)):
                label=lable[i].cpu().numpy()
                gt= ''.join([reverse_vocab[idx] for idx in label if idx in reverse_vocab])
                pred= decodedP[i]
                if pred==gt:
                    total_correctP += 1
                min_len= min(len(pred),len(gt))

                
                correct_chars= sum(1 for p,g in zip(pred[:min_len],gt[:min_len]) if p == g)
                total_correctC += correct_chars
                totalC+= len(gt)
                totalS += 1
    plate_accuracy= (total_correctP / totalS) * 100
    char_accuracy= (total_correctC / totalC) * 100
    return plate_accuracy,char_accuracy


In [ ]:
class PlateDataset(Dataset):
    def __init__(self,df,transform=None):
        self.df=df
        self.transform=transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        row=self.df.iloc[idx]


        image_data=row['image_path']['bytes']
        
        image=Image.open(io.BytesIO(image_data))
        image=image.convert('L')
        label=row['label']
        label_encoded=[vocab[char] for char in label]
        label_encoded=torch.tensor(label_encoded,dtype=torch.int64)
        if self.transform:
            image=self.transform(image)
        return image,label_encoded

In [ ]:
dataset_folder='./persian-license-plate-v1'
trainP=os.path.join(dataset_folder,'data','train-00000-of-00001.parquet')
valP=os.path.join(dataset_folder,'data','validation-00000-of-00001.parquet')
testP=os.path.join(dataset_folder,'data','test-00000-of-00001.parquet')

In [ ]:
print("لود وکبیولری...")
with open(vocab_path,'r',encoding='utf-8') as f:
    vocab=json.load(f)
reverse_vocab={v: k for k,v in vocab.items()}
char_map={
    '۰': '0','۱': '1','۲': '2'
    ,'۳': '3','۴': '4',
    '۵': '5','۶': '6','۷': '7'
    ,'۸': '8','۹': '9',
    'ي': 'ی'
}

In [ ]:
train_transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64,128)),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2),
    transforms.GaussianBlur(kernel_size=5,sigma=0.5),
    transforms.RandomAffine(degrees=0,shear=5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])
val_transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

In [ ]:
train_df=pd.read_parquet(trainP)
val_df=pd.read_parquet(valP)
test_df=pd.read_parquet(testP)
def map_and_filter_labels(label) :
    mapped_label=''.join(char_map.get(char,char) for char in label)
    return all(char in vocab for char in mapped_label),mapped_label
for df,name in [(train_df,'train'),(val_df,'validation'),(test_df,'test')]:
    valid_data=[]
    for idx,row in tqdm(df.iterrows() ,total=len(df),desc=f"پردازش لیبل‌های {name}"):
        is_valid,mapped_label=map_and_filter_labels(row['label'])
        if is_valid:
            row=row.copy()
            row['label']=mapped_label
            valid_data.append(row)
        else:
            print(f"DEL {row['label']} -> {mapped_label}")
    df=pd.DataFrame(valid_data)
    if name == 'train':
        train_df= df
    elif name== 'validation':
        val_df= df
    else:
        test_df= df



train_dataset=PlateDataset(train_df,transform=train_transform)
val_dataset=PlateDataset(val_df,transform=val_transform)
test_dataset=PlateDataset(test_df,transform=val_transform)
def collate_fn(batch):
    IMGE,lable=zip(*batch)
    IMGE=torch.stack(IMGE)
    return IMGE,lable

trainL=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=2,collate_fn=collate_fn)
valL=DataLoader(val_dataset,batch_size=batch_size,shuffle=False,num_workers=2,collate_fn=collate_fn)
testL=DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=2,collate_fn=collate_fn)

class PrintShape(nn.Module):
    def __init__(self,layer_name):
        super(PrintShape,self).__init__()
        self.layer_name=layer_name
    def forward(self,x):
        return x
num_classes=len(vocab)   
M=CRNN(num_classes,hidden_size=256).to(device)
print(f"Load {best_model_path}")
M.load_state_dict(torch.load(best_model_path))
M.to(device)
criterion=nn.CTCLoss(blank=0,zero_infinity=True)
optimizer=optim.Adam(M.parameters(),lr=learning_rate,weight_decay=weight_decay)
scheduler=optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.5,patience=5)
def decode_predictions(outputs,reverse_vocab):
    _,max_indices=torch.max(outputs,dim=2)
    decoded=[]
    for seq in max_indices:
        prev=None
        decoded_seq=[]
        for idx in seq:
            idx=idx.item()
            if idx != 0 and idx != prev:  
                if idx in reverse_vocab:
                    decoded_seq.append(reverse_vocab[idx])
            prev=idx
        decoded.append(''.join(decoded_seq))
    return decoded
def train_epoch(M,loader,criterion,optimizer):
    M.train()
    total_loss=0
    for IMGE,lable in tqdm(loader,desc="آموزش"):
        IMGE=IMGE.to(device)
        max_len=max([len(label) for label in lable])
        padded_labels=torch.zeros(len(lable),max_len,dtype=torch.long).to(device)
        for i,label in enumerate(lable):
            padded_labels[i,:len(label)]=label
        optimizer.zero_grad()
        outputs=M(IMGE).log_softmax(2)
        batch_size=IMGE.size(0)
        input_lengths=torch.full((batch_size,),outputs.size(1),dtype=torch.long)
        target_lengths=torch.tensor([len(label) for label in lable],dtype=torch.long)
        loss=criterion(outputs.permute(1,0,2),padded_labels,input_lengths,target_lengths)
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"invalid loss: {loss.item()}")
            continue
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(M.parameters(),max_norm=5.0)
        optimizer.step()
    return total_loss / len(loader)
best_val_loss=float('inf')
patienceC=0
for epoch in range(max_epochs):
    train_loss=train_epoch(M,trainL,criterion,optimizer)
    val_loss=validate_epoch(M,valL,criterion)
    train_plate_acc,train_char_acc=compute_accuracy(M,trainL,reverse_vocab)
    val_plate_acc,val_char_acc=compute_accuracy(M,valL,reverse_vocab)
    print(f"Epoch {epoch+1}/{max_epochs}:")
    print(f"Train Loss: {train_loss:.4f},Val Loss: {val_loss:.4f}")
    print(f"Train Plate Acc: {train_plate_acc:.2f}%,Train Char Acc: {train_char_acc:.2f}%")
    print(f"Val Plate Acc: {val_plate_acc:.2f}%,Val Char Acc: {val_char_acc:.2f}%")
    scheduler.step(val_loss)
    if val_loss < best_val_loss:
        best_val_loss=val_loss
        patienceC=0
        torch.save(M.state_dict(),finetuned_model_path)
        print(f"SAVE: {finetuned_model_path}")
    else:
        patienceC += 1
        if patienceC>=patience:
            print("Early Stopping!")
            break
test_plate_acc,test_char_acc=compute_accuracy(M,testL,reverse_vocab)
print(f"Test Plate Accuracy: {test_plate_acc:.2f}%")
print(f"Test Character Accuracy: {test_char_acc:.2f}%")
print("fine-tuning END!!!!")